In [20]:
import os
import sys
from test import test
from upload import retrieve_file
import warnings



ImportError: cannot import name 'retrieve_file' from 'upload' (/Users/sophiaepst/github/cphy_final/upload.py)

In [ ]:
class ParameterRecovery:
    def __init__(self, known_params, unknown_params, neuron_type= 'L5PC', num_ap = 1, V_data = None, I_data = None, t_data = None): 
        '''Args: 
            known_params (dict): known parameters and their values
            unknown_parameters (dict): unknown parameters and their intial guesses
            neuron_types (str): 
            num_ap (int):
            V_data ():
            I_data ():
            t_data ():
            '''
        self.known_params = known_params
        self.unknown_params = unknown_params
        self.params = known_params + unknown_params
        self.inst_file = retrieve_file(neuron_type= 'L5PC', num_ap = 1, V_data = None, I_data = None, t_data = None)
        self.guess = initial_guess
    if 'stim' in self.unknown_params: 
        warnings.warn('If input stimuli is unknown, can only invert for features of the stimuli, all other parameters will be assumed to be known and set to default')
    
    def assign_parameters(self):
        for key, value in self.known_params.items():
            globals()[key] = value
        for key, value in self.unknown_params.items():
            globals()[key] = value
    #forward model
    
    #cost function
    
    #gradient computation
    
    #find minimizer

In [11]:


class ParameterRecovery:
    def __init__(self, known_params, unknown_params, neuron_type= 'L5PC', num_ap = 1, V_data = None, I_data = None, t_data = None): 
        '''Args: 
            known_params (dict): known parameters and their values
            unknown_parameters (dict): unknown parameters and their intial guesses
            neuron_types (str): 
            num_ap (int):
            V_data ():
            I_data ():
            t_data ():
            '''
        self.known_params = known_params
        self.unknown_params = unknown_params
        self.params = known_params + unknown_params
        self.inst_file = retrieve_file(neuron_type= 'L5PC', num_ap = 1, V_data = None, I_data = None, t_data = None)
        self.guess = initial_guess
    if 'stim' in self.unknown_params: 
        warnings.warn('If input stimuli is unknown, can only invert for features of the stimuli, all other parameters will be assumed to be known and set to default')
    
    def assign_parameters(self):
        for key, value in self.known_params.items():
            globals()[key] = value
        for key, value in self.unknown_params.items():
            globals()[key] = value
    #forward model
    
    #cost function
    
    #gradient computation
    
    #find minimizer

NameError: name 'self' is not defined

In [2]:
my_dict = {"a": 10.0, "b": 12.0, "c": 10.0}

for key, value in my_dict.items():
    globals()[key] = value

# Now you can use variables a, b, and c with their respective values
print(f"a: {a}, b: {b}, c: {c}")


a: 10.0, b: 12.0, c: 10.0


In [3]:
a

10.0